# Import Libraries and Datasets

In [79]:
import pandas as pd
import numpy as np
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime as dt

In [80]:
# Importing excel files
company_incorporations_xls = pd.ExcelFile("Business Insights and Worforce Datasets - ADS/companyincorporationsvoluntarydissolutionsandcompulsorydissolutionsdataset030223.xlsx")
hr1_redundancies_xls = pd.ExcelFile("Business Insights and Worforce Datasets - ADS/hr1redundanciesdataset230223.xlsx")
online_job_adverts_xls = pd.ExcelFile("Business Insights and Worforce Datasets - ADS/onlinejobadvertestimatesdataset230223.xlsx")
uk_small_businesses_xls = pd.ExcelFile("Business Insights and Worforce Datasets - ADS/uksmallbusinesssalesandjobsdataset060123.xlsx")
vat_flash_xls = pd.ExcelFile("Business Insights and Worforce Datasets - ADS/vatflashdataset160223.xlsx")

#Importing CSV files
gdp_data = pd.read_csv("Consumer Behaviour Datasets - ADS/Figure_1__UK_Gross_domestic_product_(GDP)_is_estimated_to_have_fallen_by_0.5%_in_December_2022.csv")


In [81]:
#Reading in all excel files - specific sheets
company_incorporations_incorp = pd.read_excel(company_incorporations_xls, "Incorporations")
company_incorporations_vol = pd.read_excel(company_incorporations_xls, "Voluntary Dissolutions")
company_incorporations_comp = pd.read_excel(company_incorporations_xls, "Compulsory Dissolutions")

hr1_redundancies = pd.read_excel(hr1_redundancies_xls,"Potential redundancies")

online_job_adverts_category = pd.read_excel(online_job_adverts_xls,"Adverts by category Feb 2020 DD")
online_job_adverts_region = pd.read_excel(online_job_adverts_xls,"Adverts by region Feb 2020 DD")

uk_small_businesses_jobs = pd.read_excel(uk_small_businesses_xls,"Jobs by industry")
uk_small_businesses_sales = pd.read_excel(uk_small_businesses_xls,"Sales by industry")

vat_flash_turnover = pd.read_excel(vat_flash_xls, "Turnover SA MoM")
vat_flash_exp = pd.read_excel(vat_flash_xls, "Expenditure SA MoM")
vat_flash_newreporters = pd.read_excel(vat_flash_xls, "New reporters SA")
vat_flash_records = pd.read_excel(vat_flash_xls, "Record type SA")

# Creating a class to help clean data

In [82]:
class clean:
    
    def __init__ (self,df,query_dataset=False,setheader= False,header_int=0,droprows=False,rows_to_drop=[],
                  drop_cols=False,columns_to_drop=[],to_transpose=False,to_datetime=False,dayfirst=False,nancolumn=False,columns_to_keep=[],
                  converttopct=False,converttonan=False,rename_cols=False,to_append='',replace_nan=False):
        
        self.df = df
        self.query_dataset= query_dataset
        self.to_transpose = to_transpose
        self.setheader = setheader
        self.header_int = header_int
        self.droprows = droprows
        self.rows_to_drop=rows_to_drop
        self.drop_cols=drop_cols
        self.columns_to_drop=columns_to_drop
        self.to_datetime = to_datetime
        self.dayfirst=dayfirst
        self.converttopct=converttopct
        self.converttonan=converttonan
        self.nancolumn=nancolumn
        self.columns_to_keep=columns_to_keep
        self.renamecols=rename_cols
        self.toappend = to_append
        self.replace_nan = replace_nan
    
    def getnewheaders(self):
        return self.df.columns
    
    def agg_column(self):
        cols = self.getnewheaders()
        if len(cols) == 1:
            column_list = self.getnewheaders()
            agg_column = column_list[0]
        else:
            agg_column = self.getnewheaders()
        return agg_column
    
       
    def convert_to_pct(self):
        aggregate_col = self.agg_column()
        if type(aggregate_col) == str:
            for i,index in enumerate(self.df.index):
                my_str = str(self.df[aggregate_col][i])
                my_str = my_str[:-1]
                my_str = pd.to_numeric(my_str)
                self.df.loc[self.df.index == index,aggregate_col] = my_str/100
                
        else:
            column_list = self.agg_column()
            for label in column_list:
                for i,index in enumerate(self.df.index):
                    my_str = str(self.df[label][i])
                    my_str = my_str[:-1]
                    my_str = pd.to_numeric(my_str)
                    self.df.loc[self.df.index == index,label] = my_str/100
                    
                    
    def convert_to_nan(self):
        aggregate_col = self.agg_column()
        if type(aggregate_col) == str:
            self.df[aggregate_col] = pd.to_numeric(self.df[aggregate_col],errors='coerce')
                
        else:
            column_list = self.agg_column()
            for label in column_list:
                self.df[label] = pd.to_numeric(self.df[label],errors='coerce')
            
                    
    def resampleandchangeformat(self):
        self.df = self.df.resample('M',kind='timestamp').mean()
        self.df.index = self.df.index.strftime('%b %Y')
        return self.df

                
    def final_clean(self):
        if self.query_dataset == True:
            self.df = self.df.query("Name == 'United Kingdom'")
        if self.setheader == True:
            header_array = self.df.iloc[self.header_int].ravel()
            self.df.set_axis(header_array,axis=1,inplace=True)
        if self.droprows == True:
            self.df.drop(self.rows_to_drop,inplace=True)
        self.df.set_index(self.df.columns[0] ,inplace=True)
        self.df.index.names = ['Date']
        if self.drop_cols==True:
             self.df.drop(columns=self.columns_to_drop,inplace=True)
        if self.to_transpose == True:
            self.df = self.df.T
        if self.to_datetime == True and self.dayfirst==True:
            self.df.index = pd.to_datetime(self.df.index,dayfirst=True)
        else:
            self.df.index = pd.to_datetime(self.df.index)
        if self.replace_nan == True:
            self.df.iloc[:,0] = self.df.iloc[:,0].replace(np.nan, '0%')
        if self.nancolumn==True:
            self.df = self.df.loc[:,self.columns_to_keep]
        if self.converttopct==True:
            self.convert_to_pct()
        if self.converttonan==True:
            self.convert_to_nan()
        self.df= self.resampleandchangeformat()
        a = dt.strptime(self.df.index[0], '%b %Y')
        b = dt.strptime('Jan 2020', '%b %Y')
        if a < b:
            self.df = self.df['Jan 2020':'Dec 2022']
        else:
            self.df = self.df[self.df.index[0]:]
        if self.renamecols == True:
            column_list = self.getnewheaders()
            for label in column_list:
                to_append = '_' + self.toappend
                new_column_name = str(label)+to_append
                self.df.rename(columns={label:new_column_name},inplace=True)
        return self.df
        

# Dataset cleaning

In [83]:
#Before cleaning
company_incorporations_incorp.head()

,Number of weekly company incorporation applications recorded by Companies House[note 1],Unnamed: 1,Unnamed: 2
0,This worksheet contains one table.,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN
2,Companies House,NaN,NaN
3,Week ending to,Weekly,4 week average
4,2019-01-04 00:00:00,5857,[x]


In [84]:
#Setting required variables
cii_header=3
cii_droprows = [0,1,2,3]
cii_dropcols = ['4 week average']

#class instance
dataset_cleaning = clean(company_incorporations_incorp,setheader= True,header_int=cii_header,droprows=True,rows_to_drop=cii_droprows,
                  drop_cols=True,columns_to_drop=cii_dropcols,rename_cols=True,to_append='Incorporations')
company_incorporations_incorp = dataset_cleaning.final_clean()

#After cleaning
company_incorporations_incorp.head()

,Weekly_Incorporations
Date,
Jan 2020,12162.20
Feb 2020,14276.00
Mar 2020,12359.75
Apr 2020,9041.25
May 2020,12878.40


In [85]:
#Before cleaning
company_incorporations_vol.head()

,Number of weekly company voluntary dissolution applications recorded by Companies House [note 1][note 2],Unnamed: 1,Unnamed: 2
0,This worksheet contains one table. Some cells ...,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN
2,Companies House,NaN,NaN
3,Week ending to,Weekly,4 week average
4,2019-01-04 00:00:00,3095,[x]


In [86]:
#Setting required variables
civ_header=3
civ_droprows = [0,1,2,3]
civ_dropcols = ['4 week average']

#class instance
dataset_cleaning = clean(company_incorporations_vol,setheader= True,header_int=civ_header,droprows=True,rows_to_drop=civ_droprows,
                  drop_cols=True,columns_to_drop=cii_dropcols,rename_cols=True,to_append='Voluntary_Dissolutions')
company_incorporations_vol = dataset_cleaning.final_clean()

#After cleaning
company_incorporations_vol.head()

,Weekly_Voluntary_Dissolutions
Date,
Jan 2020,5901.40
Feb 2020,6094.75
Mar 2020,5683.50
Apr 2020,4183.50
May 2020,4476.00


In [87]:
#Before cleaning
company_incorporations_comp.head()

,Number of weekly compulsory company dissolutions recorded by Companies House [note 3][note 4][note 5],Unnamed: 1,Unnamed: 2,Unnamed: 3
0,This worksheet contains one table.,NaN,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN,NaN
2,Companies House,NaN,NaN,NaN
3,Week ending to,Weekly,4 week average,Cumulative
4,2018-01-02 00:00:00,10164,[x],10164


In [88]:
#Setting required variables
cic_header=3
cic_droprows = [0,1,2,3]
cic_dropcols = ['4 week average','Cumulative']

#class instance
dataset_cleaning = clean(company_incorporations_comp,setheader= True,header_int=cic_header,droprows=True,rows_to_drop=cic_droprows,
                  drop_cols=True,columns_to_drop=cic_dropcols,converttonan=True,rename_cols=True,to_append='Compulsory_Dissolutions')
company_incorporations_comp = dataset_cleaning.final_clean()

#After cleaning
company_incorporations_comp.head()

,Weekly_Compulsory_Dissolutions
Date,
Jan 2020,5827.50
Feb 2020,8851.00
Mar 2020,7569.00
Apr 2020,2925.25
May 2020,5.50


In [89]:
#Before cleaning
hr1_redundancies.head()

,Number of potential redundancies from HR1 forms and number of employers proposing redundancies,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,,.1,.2,.3,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Date,Potential redundancies,Employers proposing redundancies,Potential redundancies 4-week rolling average,Potential redundancies \n4-week rolling averag...,Employers proposing redundancies\n4-week rolli...,Employers proposing redundancies\n4-week rolli...,,,NaN,,NaN,
1,2019-04-07 00:00:00,3115,62,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
2,2019-04-14 00:00:00,*,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-04-21 00:00:00,2409,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-04-28 00:00:00,3291,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
#Setting required variables
hr_header=0
hr_droprows = [0]
hr_dropcols = ['Potential redundancies 4-week rolling average','Potential redundancies \n4-week rolling average \n(indexed: 100 = weekly average from week ending 21 Apr 19 to week ending 23 Feb 20)','Employers proposing redundancies\n4-week rolling average','Employers proposing redundancies\n4-week rolling average (indexed: 100 = weekly average from week ending 21 Apr 19 to week ending 23 Feb 20)']

#class instance
dataset_cleaning = clean(hr1_redundancies,setheader= True,header_int=hr_header,droprows=True,rows_to_drop=hr_droprows,drop_cols=True,columns_to_drop=hr_dropcols,converttonan=True,nancolumn=True,columns_to_keep=['Potential redundancies','Employers proposing redundancies'])
hr1_redundancies = dataset_cleaning.final_clean()

#After cleaning
hr1_redundancies.head()

,Potential redundancies,Employers proposing redundancies
Date,,
Jan 2020,5198.25,50.25
Feb 2020,6139.00,73.00
Mar 2020,8397.80,77.80
Apr 2020,10488.50,58.50
May 2020,17462.80,141.40


In [91]:
#Before cleaning
online_job_adverts_category.head()

,Adverts by category indexed with February 2020 for deduplicated job adverts,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,Index 100: average February 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table. Some cells ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Some cells in the notes column of this table a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Colour is used for emphasis in this table. The...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Source: Adzuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#Setting required variables
ojac_header= 5
ojac_droprows = [0,1,2,3,4,5]
ojac_dropcols = ['All industries','Notes']

#class instance
dataset_cleaning = clean(online_job_adverts_category,setheader=True,header_int=ojac_header,droprows=True,rows_to_drop=ojac_droprows,
                         drop_cols=True,columns_to_drop=ojac_dropcols,rename_cols=True,to_append='category')
online_job_adverts_category = dataset_cleaning.final_clean()

#After cleaning
online_job_adverts_category.head()

,Accounting / Finance_category,Admin / clerical / secretarial_category,Catering and hospitality_category,Charity / voluntary_category,Construction / trades_category,Creative / design / arts & media_category,Customer service / support_category,Domestic help_category,Education_category,Energy / oil & gas_category,...,Other / general_category,Part-time / weekend_category,Property_category,Wholesale and retail_category,Sales_category,Scientific / QA_category,Transport / logistics / warehouse_category,Travel / tourism_category,Unknown_category,Healthcare and Social care_category
Date,,,,,,,,,,,,,,,,,,,,,
Jan 2020,93.060,92.180,96.120,91.440,92.380,96.820,93.960,99.060,89.320,106.240,...,120.340,94.560,89.360,96.280,92.420,91.280,109.960,111.060,126.520,94.860
Feb 2020,100.025,100.000,100.000,100.000,99.975,99.975,100.000,100.000,100.000,99.975,...,100.025,100.000,100.025,99.975,100.025,100.025,100.025,100.025,100.000,99.975
Mar 2020,96.125,88.425,72.925,88.650,95.750,90.725,89.575,103.525,99.900,87.800,...,98.200,85.425,93.400,88.675,88.225,94.225,102.700,98.575,142.875,96.300
Apr 2020,53.000,37.225,24.650,42.675,48.925,54.175,46.375,113.775,78.675,38.150,...,61.275,45.650,45.575,43.400,38.775,64.925,58.050,48.550,37.775,92.225
May 2020,30.900,18.740,13.580,28.160,26.720,37.980,26.000,80.620,61.400,25.440,...,41.820,33.700,28.700,28.680,17.980,53.360,34.360,25.700,21.540,85.660


In [93]:
#Before cleaning
online_job_adverts_region.head()

,Adverts by region indexed with February 2020 for deduplicated job adverts,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,Index 100: average February 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table. Some cells ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Some cells in the notes column of this table a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Colour is used for emphasis in this table. The...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Source: Adzuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
#Setting required variables
ojar_header= 5
ojar_droprows = [0,1,2,3,4,5]
ojar_dropcols = ['All Regions','Notes']

#class instance
dataset_cleaning = clean(online_job_adverts_region,setheader=True,header_int=ojar_header,droprows=True,rows_to_drop=ojar_droprows,
                         drop_cols=True,columns_to_drop=ojar_dropcols,rename_cols=True,to_append='region')
online_job_adverts_region = dataset_cleaning.final_clean()

#After cleaning
online_job_adverts_region.head()

,East Midlands_region,East of England_region,London_region,North East_region,North West_region,Northern Ireland_region,Scotland_region,South East_region,South West_region,Unknown_region,Wales_region,West Midlands_region,Yorkshire and The Humber_region,England_region
Date,,,,,,,,,,,,,,
Jan 2020,94.280,95.64,93.960,93.520,93.300,85.540,88.440,94.860,95.280,103.000,87.46,96.200,95.10,94.620
Feb 2020,100.025,100.00,100.000,99.975,100.000,100.000,99.975,100.000,100.025,100.000,100.00,100.000,100.00,99.975
Mar 2020,91.375,93.15,91.575,89.800,92.975,84.175,88.150,92.475,91.950,105.175,81.80,94.625,92.80,92.350
Apr 2020,56.900,58.00,56.275,58.350,57.450,56.600,48.575,55.850,55.875,73.175,52.05,56.900,56.35,56.625
May 2020,39.200,40.58,37.240,41.520,39.120,43.840,36.180,37.460,39.500,61.460,36.22,37.160,37.82,38.240


In [95]:
#Before cleaning
uk_small_businesses_jobs.head()

,UK small business jobs,Unnamed: 1,,.1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Index (100 = equivalent month of 2019),NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
1,Month,Manufacturing,Construction,Retail Trade,Accommodation and Food Services,Information Media and Telecommunications,"Rental, Hiring and Real Estate Services","Professional, Scientific and Technical Services",Administrative and Support Services,Other Services,Health Care and Social Assistance
2,2019-01-01 00:00:00,100,100,100,100,100,100,100,100,100,100
3,2019-02-01 00:00:00,100,100,100,100,100,100,100,100,100,100
4,2019-03-01 00:00:00,100,100,100,100,100,100,100,100,100,100


In [96]:
#Setting required variables
sbj_header=1
sbj_droprows = [0,1]

#class instance
dataset_cleaning = clean(uk_small_businesses_jobs,setheader=True,header_int=sbj_header,droprows=True,
                         rows_to_drop=sbj_droprows,rename_cols=True,to_append='Jobs')
uk_small_businesses_jobs = dataset_cleaning.final_clean()

#After cleaning
uk_small_businesses_jobs.head()

,Manufacturing_Jobs,Construction_Jobs,Retail Trade_Jobs,Accommodation and Food Services_Jobs,Information Media and Telecommunications_Jobs,"Rental, Hiring and Real Estate Services_Jobs","Professional, Scientific and Technical Services_Jobs",Administrative and Support Services_Jobs,Other Services_Jobs,Health Care and Social Assistance_Jobs
Date,,,,,,,,,,
Jan 2020,107.4,99.5,100.2,98.1,110.9,101.2,105.0,103.7,101.9,109.7
Feb 2020,105.6,97.2,99.4,98.2,110.4,101.1,104.6,101.0,101.2,110.7
Mar 2020,99.8,97.4,98.9,87.1,110.7,94.8,102.1,95.9,93.0,101.0
Apr 2020,82.3,73.8,76.9,44.5,100.7,66.4,90.4,66.4,75.4,80.0
May 2020,79.0,73.9,78.1,36.9,97.8,62.6,88.0,74.5,69.7,72.3


In [97]:
#Before cleaning
uk_small_businesses_sales.head()

,UK small business sales,Unnamed: 1,Unnamed: 2,,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,.1,Unnamed: 9,Unnamed: 10
0,Index (100 = equivalent month of 2019),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Month,Manufacturing,Construction,Retail Trade,Accommodation and Food Services,Information Media and Telecommunications,"Rental, Hiring and Real Estate Services","Professional, Scientific and Technical Services",Administrative and Support Services,Other Services,Health Care and Social Assistance
2,2019-01-01 00:00:00,100,100,100,100,100,100,100,100,100,100
3,2019-02-01 00:00:00,100,100,100,100,100,100,100,100,100,100
4,2019-03-01 00:00:00,100,100,100,100,100,100,100,100,100,100


In [98]:
#Setting required variables
sbs_header=1
sbs_droprows = [0,1]

#class instance
dataset_cleaning = clean(uk_small_businesses_sales,setheader=True,header_int=sbs_header,
                         droprows=True,rows_to_drop=sbs_droprows,rename_cols=True,to_append='Sales')
uk_small_businesses_sales = dataset_cleaning.final_clean()

#After cleaning
uk_small_businesses_sales.head()

,Manufacturing_Sales,Construction_Sales,Retail Trade_Sales,Accommodation and Food Services_Sales,Information Media and Telecommunications_Sales,"Rental, Hiring and Real Estate Services_Sales","Professional, Scientific and Technical Services_Sales",Administrative and Support Services_Sales,Other Services_Sales,Health Care and Social Assistance_Sales
Date,,,,,,,,,,
Jan 2020,105.1,104.3,106.5,105.4,112.1,108.7,108.7,103.1,105.4,109.6
Feb 2020,105.8,103.4,107.8,104.3,108.8,105.2,106.0,102.0,110.0,109.3
Mar 2020,97.4,102.2,100.6,65.7,110.5,96.6,105.4,97.0,92.6,106.4
Apr 2020,67.5,56.4,86.6,39.2,87.8,76.7,84.7,68.2,69.2,95.6
May 2020,72.2,59.1,91.9,39.5,82.5,74.7,77.0,63.7,63.9,90.8


In [99]:
#Before cleaning
vat_flash_turnover.head()

,"Diffusion index, turnover, day 7, seasonally adjusted, Month on Month",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[c] = Data have been supressed for disclosure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date,Total (A-T),Production (B-E),Services (G-T),"Agriculture, forestry and fishing (A)",Mining and quarrying (B),Manufacturing (C),"Electricity, gas, steam and air (D)","Water supply, sewerage, etc (E)",Construction (F),...,Information and communication (J),Financial and insurance activities (K),Real estate activities (L),"Professional, scientific and technical activit...",Administrative and support activities (N),Public administration and defence (O),Education (P),Human health and social work (Q),"Arts, entertainmant and recreation (R)",Other service activities (S)


In [100]:
#Setting required variables
vft_header=4
vft_droprows = [0,1,2,3,4]
vft_dropcols = ['Total (A-T)']

#class instance
dataset_cleaning = clean(vat_flash_turnover,setheader=True,header_int=vft_header,droprows=True,rows_to_drop=vft_droprows,
                         drop_cols=True,columns_to_drop=vft_dropcols,converttonan=True,rename_cols=True,to_append='Turnover')
vat_flash_turnover = dataset_cleaning.final_clean()

#After cleaning
vat_flash_turnover.head()

,Production (B-E)_Turnover,Services (G-T)_Turnover,"Agriculture, forestry and fishing (A)_Turnover",Mining and quarrying (B)_Turnover,Manufacturing (C)_Turnover,"Electricity, gas, steam and air (D)_Turnover","Water supply, sewerage, etc (E)_Turnover",Construction (F)_Turnover,Wholesale and retail; repair of motor vehicles (G)_Turnover,Transport and storage (H)_Turnover,...,Information and communication (J)_Turnover,Financial and insurance activities (K)_Turnover,Real estate activities (L)_Turnover,"Professional, scientific and technical activities (M)_Turnover",Administrative and support activities (N)_Turnover,Public administration and defence (O)_Turnover,Education (P)_Turnover,Human health and social work (Q)_Turnover,"Arts, entertainmant and recreation (R)_Turnover",Other service activities (S)_Turnover
Date,,,,,,,,,,,,,,,,,,,,,
Jan 2020,0.02,0.00,0.03,NaN,0.04,NaN,NaN,0.04,0.03,0.06,...,0.02,NaN,0.03,-0.03,0.00,NaN,0.00,-0.17,-0.05,-0.06
Feb 2020,-0.02,-0.05,0.00,NaN,-0.03,NaN,NaN,-0.01,-0.11,-0.01,...,-0.03,NaN,-0.05,-0.06,-0.04,NaN,-0.06,-0.02,-0.06,0.09
Mar 2020,-0.26,-0.18,-0.02,NaN,-0.30,NaN,NaN,-0.17,-0.06,-0.43,...,-0.18,NaN,-0.12,-0.06,-0.28,NaN,-0.43,-0.06,-0.18,-0.05
Apr 2020,-0.30,-0.19,-0.09,NaN,-0.28,NaN,NaN,-0.31,-0.20,-0.35,...,-0.24,NaN,-0.13,-0.22,-0.24,NaN,-0.17,0.18,-0.44,-0.48
May 2020,0.06,-0.09,0.05,NaN,0.06,NaN,NaN,0.04,0.09,-0.18,...,0.05,NaN,-0.18,-0.06,0.01,NaN,0.00,-0.36,-0.09,0.03


In [101]:
#Before cleaning
vat_flash_exp.head()

,"Diffusion index, expenditure, day 7, seasonally adjusted, Month on Month",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[c] = Data have been supressed for disclosure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date,Total (A-T),Production (B-E),Services (G-T),"Agriculture, forestry and fishing (A)",Mining and quarrying (B),Manufacturing (C),"Electricity, gas, steam and air (D)","Water supply, sewerage, etc (E)",Construction (F),...,Information and communication (J),Financial and insurance activities (K),Real estate activities (L),"Professional, scientific and technical activit...",Administrative and support activities (N),Public administration and defence (O),Education (P),Human health and social work (Q),"Arts, entertainmant and recreation (R)",Other service activities (S)


In [102]:
#Setting required variables
vfe_header=4
vfe_droprows = [0,1,2,3,4]
vfe_dropcols = ['Total (A-T)']

#class instance
dataset_cleaning = clean(vat_flash_exp,setheader=True,header_int=vfe_header,droprows=True,rows_to_drop=vfe_droprows,
                         drop_cols=True,columns_to_drop=vfe_dropcols,converttonan=True,rename_cols=True,to_append='Exp')
vat_flash_exp = dataset_cleaning.final_clean()

#After cleaning
vat_flash_exp.head()

,Production (B-E)_Exp,Services (G-T)_Exp,"Agriculture, forestry and fishing (A)_Exp",Mining and quarrying (B)_Exp,Manufacturing (C)_Exp,"Electricity, gas, steam and air (D)_Exp","Water supply, sewerage, etc (E)_Exp",Construction (F)_Exp,Wholesale and retail; repair of motor vehicles (G)_Exp,Transport and storage (H)_Exp,...,Information and communication (J)_Exp,Financial and insurance activities (K)_Exp,Real estate activities (L)_Exp,"Professional, scientific and technical activities (M)_Exp",Administrative and support activities (N)_Exp,Public administration and defence (O)_Exp,Education (P)_Exp,Human health and social work (Q)_Exp,"Arts, entertainmant and recreation (R)_Exp",Other service activities (S)_Exp
Date,,,,,,,,,,,,,,,,,,,,,
Jan 2020,0.06,0.03,0.05,NaN,0.05,0.13,NaN,0.08,0.02,-0.03,...,-0.03,NaN,0.01,0.11,0.16,NaN,0.03,0.05,-0.07,-0.10
Feb 2020,-0.01,-0.01,0.01,NaN,0.00,-0.07,NaN,0.00,-0.04,-0.03,...,0.03,NaN,0.05,-0.04,-0.03,NaN,-0.10,0.00,-0.20,0.13
Mar 2020,-0.12,-0.12,0.00,NaN,-0.14,-0.01,NaN,-0.19,-0.05,-0.34,...,-0.09,NaN,0.03,-0.18,-0.17,NaN,-0.43,0.02,-0.16,-0.37
Apr 2020,-0.22,-0.24,0.02,NaN,-0.27,0.10,NaN,-0.28,-0.26,-0.52,...,-0.23,NaN,-0.24,-0.40,-0.17,NaN,-0.10,0.02,-0.29,-0.27
May 2020,0.04,0.00,0.00,NaN,0.06,-0.11,NaN,0.08,0.06,-0.06,...,0.01,NaN,-0.01,0.19,-0.02,NaN,-0.14,-0.09,0.08,0.04


In [103]:
#Before cleaning
vat_flash_newreporters.head()

,"New VAT reporters, seasonally adjusted",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[c] = Data have been supressed for disclosure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date,Total (A-T),Production (B-E),Services (G-T),"Agriculture, forestry and fishing (A)",Mining and quarrying (B),Manufacturing (C),"Electricity, gas, steam and air (D)","Water supply, sewerage, etc (E)",Construction (F),...,Information and communication (J),Financial and insurance activities (K),Real estate activities (L),"Professional, scientific and technical activit...",Administrative and support activities (N),Public administration and defence (O),Education (P),Human health and social work (Q),"Arts, entertainmant and recreation (R)",Other service activities (S)


In [104]:
#Setting required variables
vfn_header=4
vfn_droprows = [0,1,2,3,4]
vfn_dropcols = ['Total (A-T)']

#class instance
dataset_cleaning = clean(vat_flash_newreporters,setheader=True,header_int=vfn_header,droprows=True,rows_to_drop=vfn_droprows,
                         drop_cols=True,columns_to_drop=vfn_dropcols,converttonan=True,rename_cols=True,to_append='Newreporters')
vat_flash_newreporters = dataset_cleaning.final_clean()

#After cleaning
vat_flash_newreporters.head()

,Production (B-E)_Newreporters,Services (G-T)_Newreporters,"Agriculture, forestry and fishing (A)_Newreporters",Mining and quarrying (B)_Newreporters,Manufacturing (C)_Newreporters,"Electricity, gas, steam and air (D)_Newreporters","Water supply, sewerage, etc (E)_Newreporters",Construction (F)_Newreporters,Wholesale and retail; repair of motor vehicles (G)_Newreporters,Transport and storage (H)_Newreporters,...,Information and communication (J)_Newreporters,Financial and insurance activities (K)_Newreporters,Real estate activities (L)_Newreporters,"Professional, scientific and technical activities (M)_Newreporters",Administrative and support activities (N)_Newreporters,Public administration and defence (O)_Newreporters,Education (P)_Newreporters,Human health and social work (Q)_Newreporters,"Arts, entertainmant and recreation (R)_Newreporters",Other service activities (S)_Newreporters
Date,,,,,,,,,,,,,,,,,,,,,
Jan 2020,1100.0,15430.0,280.0,NaN,970.0,NaN,NaN,2790.0,4420.0,1540.0,...,1780.0,190.0,590.0,3000.0,1310.0,NaN,220.0,NaN,430.0,290.0
Feb 2020,1010.0,16380.0,260.0,NaN,840.0,NaN,NaN,3020.0,4260.0,2210.0,...,1760.0,190.0,610.0,2940.0,1360.0,NaN,180.0,NaN,470.0,320.0
Mar 2020,910.0,16630.0,270.0,NaN,810.0,NaN,NaN,2850.0,4220.0,2890.0,...,1590.0,140.0,520.0,2810.0,1310.0,NaN,160.0,NaN,480.0,290.0
Apr 2020,740.0,11330.0,230.0,NaN,610.0,NaN,NaN,2200.0,3260.0,1330.0,...,1260.0,170.0,510.0,2230.0,910.0,NaN,160.0,NaN,420.0,270.0
May 2020,750.0,12060.0,250.0,NaN,630.0,NaN,NaN,2310.0,3410.0,1550.0,...,1250.0,170.0,490.0,2350.0,990.0,NaN,150.0,NaN,400.0,260.0


In [105]:
#Before cleaning
vat_flash_records.head()

,"Number of reporters by record type, seasonally adjusted",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88
0,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[x] = Data are unavailable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[c] = Data have been supressed for disclosure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date,Total - Tax due,Total - Repayment claim,Total - Re-input tax due,Total - Re-input repayment claim,Production - Tax due,Production - Repayment claim,Production - Re-input tax due,Production - Re-input repayment claim,Services - Tax due,...,Q - Re-input tax due,Q - Re-input repayment claim,R - Tax due,R - Repayment claim,R - Re-input tax due,R - Re-input repayment claim,S - Tax due,S - Repayment claim,S - Re-input tax due,S - Re-input repayment claim


In [106]:
#Setting required variables
vfr_header=4
vfr_droprows = [0,1,2,3,4]
vfr_dropcols = ['Total - Tax due','Total - Repayment claim','Total - Re-input tax due','Total - Re-input repayment claim']

#class instance
dataset_cleaning = clean(vat_flash_records,setheader=True,header_int=vfr_header,droprows=True,rows_to_drop=vfr_droprows,
                         drop_cols=True,columns_to_drop=vfr_dropcols,converttonan=True)
vat_flash_records = dataset_cleaning.final_clean()

#After cleaning
vat_flash_records.head()

,Production - Tax due,Production - Repayment claim,Production - Re-input tax due,Production - Re-input repayment claim,Services - Tax due,Services - Repayment claim,Services - Re-input tax due,Services - Re-input repayment claim,A - Tax due,A - Repayment claim,...,Q - Re-input tax due,Q - Re-input repayment claim,R - Tax due,R - Repayment claim,R - Re-input tax due,R - Re-input repayment claim,S - Tax due,S - Repayment claim,S - Re-input tax due,S - Re-input repayment claim
Date,,,,,,,,,,,,,,,,,,,,,
Jan 2020,33050.0,13560.0,NaN,NaN,393520.0,98320.0,690.0,NaN,8920.0,48020.0,...,NaN,NaN,11870.0,5340.0,NaN,NaN,13280.0,2370.0,NaN,NaN
Feb 2020,34090.0,13770.0,NaN,NaN,402550.0,99830.0,650.0,NaN,9120.0,48210.0,...,NaN,NaN,12250.0,5340.0,NaN,NaN,13480.0,2330.0,NaN,NaN
Mar 2020,31330.0,13490.0,NaN,NaN,356320.0,95310.0,520.0,NaN,8260.0,46040.0,...,NaN,NaN,11250.0,5260.0,NaN,NaN,12200.0,2310.0,NaN,NaN
Apr 2020,29970.0,13110.0,NaN,NaN,354250.0,95360.0,470.0,NaN,8430.0,47070.0,...,NaN,NaN,11530.0,5220.0,NaN,NaN,12710.0,2170.0,NaN,NaN
May 2020,32320.0,12400.0,NaN,NaN,365400.0,91240.0,540.0,NaN,8410.0,45720.0,...,NaN,NaN,11370.0,5400.0,NaN,NaN,12670.0,2250.0,NaN,NaN


In [107]:
#Before cleaning
gdp_data.head()

,Figure 1: UK Gross domestic product (GDP) is estimated to have fallen by 0.5% in December 2022,Unnamed: 1
0,"Monthly index, January 2007 to December 2022, UK",NaN
1,NaN,NaN
2,Notes,NaN
3,Unit,Office for National Statistics – GDP monthly e...
4,NaN,NaN


In [108]:
#GDP cleaning

#Setting required variables
gdp_header=5
gdp_droprows = [0,1,2,3,4,5]

#class instance
dataset_cleaning = clean(gdp_data,setheader= True,header_int=gdp_header,droprows=True,rows_to_drop=gdp_droprows,to_datetime=True)
gdp_data = dataset_cleaning.final_clean()

#After cleaning
gdp_data.head()

,Monthly GDP
Date,
Jan 2020,100.7
Feb 2020,100.2
Mar 2020,93.1
Apr 2020,73.7
May 2020,76.0


# Merging Datasets

In [109]:
#Datasets list including the GDP data
list_datasets=[gdp_data,company_incorporations_incorp,company_incorporations_vol,company_incorporations_comp,hr1_redundancies,online_job_adverts_category,online_job_adverts_region,uk_small_businesses_jobs,uk_small_businesses_sales,vat_flash_turnover,vat_flash_exp,vat_flash_newreporters,vat_flash_records]


In [110]:
#Merge datasets 
business_insights = pd.concat(list_datasets ,join = 'outer',axis=1)

In [111]:
business_insights.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, Jan 2020 to Dec 2022
Data columns (total 216 columns):
 #    Column                                                              Non-Null Count  Dtype  
---   ------                                                              --------------  -----  
 0    Monthly GDP                                                         36 non-null     float64
 1    Weekly_Incorporations                                               36 non-null     float64
 2    Weekly_Voluntary_Dissolutions                                       36 non-null     float64
 3    Weekly_Compulsory_Dissolutions                                      36 non-null     float64
 4    Potential redundancies                                              36 non-null     float64
 5    Employers proposing redundancies                                    36 non-null     float64
 6    Accounting / Finance_category                                       36 non-null     float64
 7    

In [112]:
business_insights.to_csv(r'Merged_Business_Insights.csv')

# Missing Values

In [117]:
#Define target variable and predictor variable
y = business_insights.loc['Jan 2020':'Dec 2022','Monthly GDP']
X = business_insights.drop(columns='Monthly GDP')
X = X.loc['Jan 2020':'Dec 2022',:]

In [118]:
#Create function for imputation of missing values usin KNN
def missingvalues(X):
    # check for columns that have all null values and convert to a list
    empty_train_columns = X.columns[X.isnull().all()].tolist()
    #drop those columns from the dataset
    X = X.drop(empty_train_columns, axis=1)
    #Impute missing values
    KNN_imputer = KNNImputer(n_neighbors=3)
    #Create new dataset
    X_new = pd.DataFrame(data=KNN_imputer.fit_transform(X), columns=X.columns)
    return X_new

X_new = missingvalues(X)

In [418]:
#create corr mask for upper triangle
corr_X = X_new.corr().abs()
mask = np.triu(np.ones_like(corr_X, dtype=bool))
tri_X = corr_X.mask(mask)
tri_X.head()

,Weekly_Incorporations,Weekly_Voluntary_Dissolutions,Weekly_Compulsory_Dissolutions,Potential redundancies,Employers proposing redundancies,Accounting / Finance_category,Admin / clerical / secretarial_category,Catering and hospitality_category,Charity / voluntary_category,Construction / trades_category,...,N - Tax due,N - Repayment claim,P - Tax due,P - Repayment claim,Q - Tax due,Q - Repayment claim,R - Tax due,R - Repayment claim,S - Tax due,S - Repayment claim
Weekly_Incorporations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Weekly_Voluntary_Dissolutions,0.260258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Weekly_Compulsory_Dissolutions,0.125277,0.428231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Potential redundancies,0.335835,0.546319,0.627100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Employers proposing redundancies,0.436100,0.512191,0.641606,0.975173,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [419]:
#Find columns to be dropped
to_drop = [c for c in tri_X.columns if any(tri_X[c] > 0.8)]
print(to_drop)

['Weekly_Voluntary_Dissolutions', 'Potential redundancies', 'Accounting / Finance_category', 'Admin / clerical / secretarial_category', 'Catering and hospitality_category', 'Charity / voluntary_category', 'Construction / trades_category', 'Creative / design / arts & media_category', 'Customer service / support_category', 'Domestic help_category', 'Education_category', 'Energy / oil & gas_category', 'Engineering_category', 'Facilities / maintenance_category', 'Graduate_category', 'HR & recruitment_category', 'IT / Computing / Software_category', 'Legal_category', 'Management / exec / consulting_category', 'Manufacturing_category', 'Marketing / advertising / PR_category', 'Other / general_category', 'Part-time / weekend_category', 'Property_category', 'Wholesale and retail_category', 'Sales_category', 'Scientific / QA_category', 'Transport / logistics / warehouse_category', 'Travel / tourism_category', 'East Midlands_region', 'East of England_region', 'London_region', 'North East_region'

In [420]:
#Drop Columns
X_reduced = X_new.drop(columns = to_drop)
X_reduced.head()

,Weekly_Incorporations,Weekly_Compulsory_Dissolutions,Employers proposing redundancies,Unknown_category,Healthcare and Social care_category,England_region,Health Care and Social Assistance_Sales,"Agriculture, forestry and fishing (A)_Turnover",Manufacturing (C)_Turnover,Accommodation and food services (I)_Turnover,...,G - Tax due,G - Re-input tax due,H - Tax due,L - Tax due,M - Tax due,N - Tax due,N - Repayment claim,Q - Tax due,Q - Repayment claim,S - Repayment claim
0,12162.20,5827.50,50.25,126.520,94.860,94.620,109.6,0.03,0.04,0.14,...,92410.0,200.0,21730.0,26270.0,95080.0,33200.0,8380.0,2140.0,1520.0,2370.0
1,14276.00,8851.00,73.00,100.000,99.975,99.975,109.3,0.00,-0.03,-0.02,...,93170.0,180.0,21020.0,25650.0,96720.0,33050.0,8650.0,2230.0,1530.0,2330.0
2,12359.75,7569.00,77.80,142.875,96.300,92.350,106.4,-0.02,-0.30,-0.59,...,93230.0,180.0,20810.0,23780.0,87150.0,31890.0,8620.0,2050.0,1470.0,2310.0
3,9041.25,2925.25,58.50,37.775,92.225,56.625,95.6,-0.09,-0.28,-0.53,...,83120.0,140.0,17610.0,23740.0,87470.0,30200.0,8400.0,2020.0,1560.0,2170.0
4,12878.40,5.50,141.40,21.540,85.660,38.240,90.8,0.05,0.06,-0.37,...,83290.0,160.0,22840.0,24850.0,88670.0,31680.0,8290.0,2280.0,1540.0,2250.0


# Feature Selection

## Ensemble

In [119]:
#Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=19042351)

#Scale data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [120]:
#Function for ensemble feature selection
def ensemble(X_train_std,X_test_std,y_train,y_test,num_votes):
    cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=2)
    #Find alpha
    lcv = LassoCV(max_iter=5000,random_state=19042351, selection='random')
    lcv.fit(X_train_std, y_train)
    lcv_alpha = lcv.alpha_
    
    #Fit Lasso
    lasso = Lasso(alpha=lcv_alpha,max_iter=5000,random_state=19042351, selection='random')
    lasso.fit(X_train_std, y_train)
    
    #Lasso features mask
    lasso_mask = lasso.coef_ != 0
    

    #Gradient boosting features mask
    rfe_gb = RFECV(estimator=GradientBoostingRegressor(random_state=19042351), step=3,min_features_to_select=1,cv=cv, verbose=0)
    rfe_gb.fit(X_train_std, y_train)
    gb_mask = rfe_gb.support_
    
    #Random forest features mask
    rfe_rf = RFECV(estimator=RandomForestRegressor(random_state=19042351), step=3,min_features_to_select=1,cv=cv, verbose=0)
    rfe_rf.fit(X_train_std, y_train)
    rf_mask = rfe_rf.support_
    
    #sum votes from all three models
    votes = np.sum([lasso_mask, rf_mask, gb_mask], axis=0)
    
    #Set Criteria for mask - 2 or more votes
    all_votes_mask = votes >= num_votes
    
    #New Dataset with selected features 
    X_selected = X_new.loc[:, all_votes_mask]
    
    list_scores = [lasso.score(X_test_std, y_test),rfe_gb.score(X_test_std, y_test),rfe_rf.score(X_test_std, y_test)]
    average_score = sum(list_scores)/len(list_scores)
    
    print(f'Number of columns before feature selection:{X_train_std.shape[1]}')
    print(f'Number of columns after feature selection:{X_selected.shape[1]}')
    print(f'Average accuracy score : {average_score:.1%}\n Lasso model : {lasso.score(X_test_std, y_test):.1%}\n Gradient Boosting model : {rfe_gb.score(X_test_std, y_test):.1%}\n Random Forest model : {rfe_rf.score(X_test_std, y_test):.1%}')

    
    print(X_selected.columns.values)
    return X_selected
    
X_BusinessInsights = ensemble(X_train_std,X_test_std,y_train,y_test,2)

Number of columns before feature selection:159
Number of columns after feature selection:73
Average accuracy score : 85.8%
 Lasso model : 81.4%
 Gradient Boosting model : 91.0%
 Random Forest model : 84.9%
['Potential redundancies' 'Employers proposing redundancies'
 'Admin / clerical / secretarial_category'
 'Catering and hospitality_category' 'Charity / voluntary_category'
 'Construction / trades_category'
 'Creative / design / arts & media_category'
 'Customer service / support_category' 'Domestic help_category'
 'Energy / oil & gas_category' 'Engineering_category'
 'Facilities / maintenance_category' 'Graduate_category'
 'Management / exec / consulting_category' 'Manufacturing_category'
 'Marketing / advertising / PR_category' 'Other / general_category'
 'Part-time / weekend_category' 'Wholesale and retail_category'
 'Transport / logistics / warehouse_category' 'Travel / tourism_category'
 'East Midlands_region' 'East of England_region' 'London_region'
 'North West_region' 'Norther

In [122]:
#Save to csv file
X_BusinessInsights.to_csv(r'X_BusinessInsights.csv',index=False)

# END